In [1]:
using POMDPs, POMDPTools, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies
using Distributions, Combinatorics, StaticArrays, D3Trees, Random
using FileIO, JLD2, TickTock

In [2]:
action=[(0,0), (0,1), (1,1)]

Number_level = 10;


In [3]:
T=zeros(Number_level,Number_level,3,100);

# Load transition probability matrix for each component
for i in 1:100
    fullname = "J:/Master's Thesis/Heterogeneous/TM_MATRIX/tm"*string(i);
    #fullname = "/Users/huhan/Library/CloudStorage/GoogleDrive-hhu98@wisc.edu/My Drive/Master's Thesis/Heterogeneous/TM_MATRIX/tm"*string(i);
    # generate_transition_matrix(Number_level, fullname) 
    Transition_matrix = load(fullname*".jld2","transition_matrix");
    # the last term in argument of T, which is written as 'i' here,
    # is the index of component in the heterogeneous system
    T[:,:,1,i].=Transition_matrix; # a=1: do not replace
    T[:,:,2,i].=Transition_matrix; # a=2: corrective replacement
    for j in 1:Number_level
        T[j,:,3,i].=Transition_matrix[1,:]; # a=3: preventive replacement
        end
    # T[Number_level,:,1,i].=Transition_matrix[1,:]
    T[Number_level,:,1,i].=0.0
    T[Number_level,Number_level,1,i]=1.0
    T[Number_level,:,2,i].=Transition_matrix[1,:]

end

In [4]:
function reward(s, a,k,s1=10, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
   if a==1 #NO REPAIR AND SYSTEM FAILURE
        if s==Number_level
           # r=failure_penalty +  setup_cost/NumberUnits + normal_operation  ;
            r=system_penalty/(limit);
            return r
        else
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r = normal_operation ;
             return r
            end
        end
    if a==2 # UNIT NOT REPAIRED UNLESS FAILURE BUT REPAIR IN SYSTEM
        if s==Number_level
             s=1
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r=failure_penalty +  setup_cost/NumberUnits + normal_operation  ;
        return r
        else
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r = normal_operation +  setup_cost/NumberUnits;
             return r
        end
    end

     if a==3 # UNIT REPAIRED AND MAINTENANCE IN SYSTEM NO SYSTEM FAILURE
        if s==Number_level
             s=1
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r=failure_penalty +  setup_cost/NumberUnits + normal_operation  ;
        return r
        else
        s1=deepcopy(s)
        s=1
        r = maintenance_penalty +  setup_cost/NumberUnits
         crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r+= normal_operation  ;
#            r*=(s1/10);
       return r
#         end
    end
    end
    
    if a==4 #UNIT REPAIRED AND MAINTENANCE IN SYSTEM with SYSTEM FAILURE FAILED UNIT TAKES Kth Component
        if s==Number_level
             s=1
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r=failure_penalty +  setup_cost/NumberUnits + normal_operation + system_penalty/(limit) ;
        return r
        else
        s1=deepcopy(s)
        s=1
        r = maintenance_penalty +  setup_cost/NumberUnits
         crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r+= normal_operation  ;
#            r*=(s1/10);
       return r
#         end
    end
    end
    
    end

reward (generic function with 3 methods)

In [5]:
function explore()
    c=[1/3,1/3,1/3]

     crd = Categorical(c);
     return rand(crd)
    end 
function generative(s, a,k, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
    if a==1
        # if s==Number_level
        #     r=failure_penalty +  setup_cost/NumberUnits + normal_operation  ;
        #     return (sp=s, r=r)
        # else
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r = normal_operation ;
             return (sp=s, r=r)
            #end
        end
    if a==2
        if s==Number_level
             s=1
            crd = Categorical(T[s,:,2,k]);
            s = rand(crd);
            #r=failure_penalty +  setup_cost/NumberUnits + normal_operation  ;
            r=failure_penalty  + normal_operation  ;
        return (sp=s, r=r)
        else
            crd = Categorical(T[s,:,2,k]);
            s = rand(crd);
            #r = normal_operation +  setup_cost/NumberUnits;
            r = normal_operation;
             return (sp=s, r=r)
        end
    end

     if a==3
        if s==Number_level
             s=1
            crd = Categorical(T[s,:,3,k]);
            s = rand(crd);
            #r=failure_penalty +  setup_cost/NumberUnits + normal_operation  ;
            r=failure_penalty + normal_operation  ;
        return (sp=s, r=r)
        else
        s1=deepcopy(s)
        s=1
        #r = maintenance_penalty +  setup_cost/NumberUnits
        r = maintenance_penalty
         crd = Categorical(T[s,:,3,k]);
            s = rand(crd);
            r+= normal_operation  ;
#            r*=(s1/10);
       return (sp=s, r=r)
#         end
    end
    end
    
    
    if a==4
        if s==Number_level
             s=1
            crd = Categorical(T[s,:,3,k]);
            s = rand(crd);
            #r=failure_penalty +  setup_cost/NumberUnits + normal_operation + system_penalty/(limit) ;
            r=failure_penalty+ normal_operation;
        return (sp=s, r=r)
        else
        s1=deepcopy(s)
        s=1
        #r = maintenance_penalty +  setup_cost/NumberUnits
        r = maintenance_penalty
         crd = Categorical(T[s,:,3,k]);
            s = rand(crd);
            r+= normal_operation  ;
#            r*=(s1/10);
       return (sp=s, r=r)
#         end
    end
    end
#     for l in 1:(NumberUnits-limit+1)
#         if a==(l+3)
#             if s==Number_level
#                  s=1
#                 crd = Categorical(T[s,:,1]);
#                 s = rand(crd);
#                 r=failure_penalty +  setup_cost/NumberUnits + normal_operation + system_penalty/(l+limit-1)  ;
#             return (sp=s, r=r)
#             else
#                     s1=deepcopy(s)
#                     s=1
#                     r = maintenance_penalty +  setup_cost/NumberUnits
#                      crd = Categorical(T[s,:,1]);
#                         s = rand(crd);
#                         r+= normal_operation  ;
#             #            r*=(s1/10);
#                    return (sp=s, r=r)
#             end
#         end
#     end
    
    end

generative (generic function with 2 methods)

In [6]:
function ValueIteration(γ, p1=0.01, d=100)
    Q=zeros((Number_level,4,NumberUnits))
#     print(Q)
    V= zeros(Float64, (Number_level,NumberUnits)).+1000
    # diff=0
    # i=0
    p=rand(Float64, (4,NumberUnits))
    for k in 1:NumberUnits
        for i in 1:10000
#     while  i<=10000
    for s in 1:Number_level
       Threads.@threads for a in 1:4
#                 println(a)
#                 println(reward(s,a))
                if a>3
                Q[s,a,k]= (reward(s,a,k)) + γ*( sum(T[s,s1,3,k]*(V[s1,k]) for (s1) in 1:Number_level ))
                else
                Q[s,a,k]= (reward(s,a,k)) + γ*( sum(T[s,s1,a,k]*(V[s1,k]) for (s1) in 1:Number_level ))
                end
        end
#             print("here")

                p[:,k].=[exp(lamb*(sum(Q[s,a3,k]-findmin(Q[s,1:4,k])[1]))) for a3 in 1:4]
                p[:,k]=p[:,k]./sum(p[:,k])
               # p1=p1./sum(p1)
#                  V[s]=sum(p1[1:3])*(p[1]*Q[s,1] + (p[2]+p[3])*max(Q[s,2],Q[s,3])) + sum(p1[4:(NumberUnits-limit+4)])*(sum(p2[b]*Q[s,b+3] for b in 1:(NumberUnits-limit+1)))
                 # V[s,k]=(p1[1]*Q[s,1,k] + p1[2]*Q[s,2,k] +p1[3]*Q[s,3,k]+p1[4]*Q[s,4,k])
                 V[s,k]=Q[s,1,k]*p[1,k]+Q[s,2,k]*p[2,k]+Q[s,3,k]*p[3,k]+Q[s,4,k]*p[4,k]
#             println(findmax(Q[s,1:4]))
#                 println(Q)
#                 V[s]=findmax(Q[s,1:4])[1]
            # p[:,k].=[exp(lamb*(sum(Q[s,1,k]-findmin(Q[s,1:3,k])[1]))),exp(lamb*(Q[s,2,k]-findmin(Q[s,1:3,k])[1])),exp(lamb*(Q[s,3,k]-findmin(Q[s,1:3,k])[1]))]

            # p[:,k]=p[:,k]./sum(p[:,k])
            # V[s,k]=Q[s,1,k]*p[1,k]+Q[s,2,k]*p[2,k]+Q[s,3,k]*p[3,k]+Q[s,4,k]*p[4,k]
                 
        end
        end
        end
       # print(V)
    return Q
    end

ValueIteration (generic function with 3 methods)

In [7]:

# function decison(Q, s)
#    a_final=[]
#    if Number_level in s
# #         print("hi")
#         d=(collect(Iterators.product((Q[s[i],2:3] for i in 1:NumberUnits)...)))
#         d1=(collect(Iterators.product((2:3 for i in 1:NumberUnits)...)))
#         min=-1e+10
#         opt=0
#         for (i,j) in enumerate(d)
#             if min < sum((j))
#                 min=sum((j))
#                 opt=i
#             end
#         end
#         return d1[opt]
#     end
#     q0=0.0
#     a_0=repeat(1:1,NumberUnits);
#     for i in 1:NumberUnits
#             q0+=Q[s[i],1]
#     end
#     d=(collect(Iterators.product((Q[s[i],2:3] for i in 1:NumberUnits)...)))
#     d1=(collect(Iterators.product((2:3 for i in 1:NumberUnits)...)))
# #     d[1]=a_0
# #     print("here")
# #     d1[1]=(1,1,1,1,1,1,1,1,1,1)
#     min=-1e+10
#     opt=0
#     for (i,j) in enumerate(d)
#         if i==1
#             if min < q0
#             min=q0
#             opt=i
#         end
#         elseif min < sum((j))
#             min=sum((j))
#             opt=i
#         end
#     end
#     if opt==1
#         return a_0
#     else
#         return d1[opt]
#     end
#     return opt
# end


function decison(Q, s,n=0.2)
   a_final=[]
    alpha=count(i->(i==Number_level), s) 
    alpha=deepcopy(limit)
   if count(i->(i==Number_level), s) >=limit
#         print("hi")
        count_1=0
        a_1=repeat(1:1,NumberUnits);
        for i in 1:NumberUnits
            a_1[i]=argmax(a->Q[s[i],a,i], [2,3])
            if s[i]==Number_level && alpha>0
                a_1[i]=4
                alpha=alpha-1
            end
        end
        return a_1
    end
    
    
    
    
    
    q0=0.0
    a_0=repeat(1:1,NumberUnits);
    for i in 1:NumberUnits
            q0+=Q[s[i],1,i]
    end
    a_1=repeat(1:1,NumberUnits);
    b=0
    for i in 1:NumberUnits
            a_1[i]=argmax(a->Q[s[i],a,i], [2,3])
            if s[i]==Number_level
                a_1[i]=3
            end
    end
    q1=0.0
    for i in 1:NumberUnits
            q1+=Q[s[i],a_1[i],i]
    end
    
    if !(3 in a_1) 
        a_1=a_0
        q1=q0
    end
    
    
     if q0>q1
            return a_0
        else
#             print("HI")
            return a_1
            
        end
end

decison (generic function with 2 methods)

In [8]:
using DataFrames
df= DataFrame(u=[],K=[],n=[],s=[],m=[],f=[],p=[],lamb=[],mean=[],std=[])

Row,u,K,n,s,m,f,p,lamb,mean,std
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any


In [9]:

cost=[ [0	-200	-100	-1200 -2000],
[0	-400	-100	-1200 -2000],
[0	-600	-100	-1200 -2000],
[0	-800	-100	-1200 -2000],
[0	-1000	-100	-1200 -2000],
[0	-1200	-100	-1200 -2000],
[0	-1400	-100	-1200 -2000],
[0	-1600	-100	-1200 -2000],
[0	-1800	-100	-1200 -2000],
[0	-2000	-100	-1200 -2000],
[0  -2400   -100    -1200 -2000],
[0  -2800   -100    -1200 -2000],
[0  -3200   -100    -1200 -2000]
] 



for units in [20,25,30,40,50,60]
for K in 2:convert(Int64, floor((units+1)/10)):convert(Int64, floor((units+1)/2))
    
global NumberUnits=convert(Int64,units)
    global limit=convert(Int64,K)
#      for lambda in [1,0.9,0.75,0.5,0.1,0.075,0.05,0.01,0.0075,0.005,0.0001]
for lambda in [10,5,1,0.5, 0.1,0.05,1e-2,0.5e-2,1e-3,0.5e-3]
            
        
        global lamb=lambda
#         global lamb1=lambda1
            
            for c in cost
        tick()
   println(units)  
             print("K : ")
                    println(K)
    global normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty=c
     println(c)   

    global NumberUnits=units
     Q=ValueIteration(.95,3000)
#                 println(Q)
            println(isnan(Q[1,1,1]))
       if sum(isnan(Q[1,1,1]))>0.0
            continue
        end

trials=10000;
simsteps = 100;
results=zeros(trials)*0.1
Threads.@threads  for h in 1:trials
s=repeat(1:1,NumberUnits)
r1=[]
a1=[]   
r=repeat(1:1,NumberUnits)*1.0
 for k in 1:simsteps
    a =decison(Q,s)
            r2=0.0
            global failed_components = 0
#     print(s)
#                         println(a)
    for i in 1:NumberUnits
#             print(generative(s[i],a[i],234))
        if s[i] == Number_level
            failed_components += 1
        end
         s[i],r[i]=generative(s[i],a[i],i,234);
          r2+=r[i]
            end 
#     println(r2)
#     println(s)
if  (2 in a) ||(3 in a) ||(4 in a) 
    # if any replacement
        r2+=setup_cost
                end
if failed_components >= limit
    r2+=system_penalty
end
    s=s;
    append!(a1,a)
    append!(r1,r2)
end
rk=0
for i in 1:simsteps
    rk=0.95*rk+r1[simsteps-i+1]
end
#     println(rk)
    results[h]=rk
#     println(results)
end
println(units)
            print("k : ")
            println(limit)
println(lambda)

println(mean(results))

println(std(results))
pushfirst!(df,[units,limit,normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty,lamb,mean(results),std(results)])
        
tock()
end
    println("-------------------------")
end
end
end
end


20
K : 2

[ Info:  started timer at: 2024-04-02T09:26:39.346



[0 -200 -100 -1200 -2000]
true
20
K : 2
[0 -400 -100 -1200 -2000]

[ Info:  started timer at: 2024-04-02T09:28:15.026



true
20
K : 2
[

...

[ Info:          163.8341864s: 2 minutes, 43 seconds, 834 milliseconds
[ Info:  started timer at: 2024-04-05T17:41:02.417


false


Excessive output truncated after 524339 bytes.

40
k : 2
0.05
-73629.08208405616
5470.268470996358


LoadError: ParseError:
[90m# Error @ [0;0m]8;;file://j:/CW-MDP revision/Code/heterogeneous/In[9]#106:1\[90mIn[9]:106:1[0;0m]8;;\
end
[48;2;120;70;70mend[0;0m
[90m└─┘ ── [0;0m[91minvalid identifier[0;0m

In [10]:
df

Row,u,K,n,s,m,f,p,lamb,mean,std
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,60.0,26.0,0.0,-3200.0,-100.0,-1200.0,-2000.0,0.0005,-1.0071e5,4417.63
2,60.0,26.0,0.0,-2800.0,-100.0,-1200.0,-2000.0,0.0005,-97848.0,4325.85
3,60.0,26.0,0.0,-2400.0,-100.0,-1200.0,-2000.0,0.0005,-94668.0,4234.43
4,60.0,26.0,0.0,-2000.0,-100.0,-1200.0,-2000.0,0.0005,-91312.3,4099.82
5,60.0,26.0,0.0,-1800.0,-100.0,-1200.0,-2000.0,0.0005,-89547.1,4020.16
6,60.0,26.0,0.0,-1600.0,-100.0,-1200.0,-2000.0,0.0005,-87686.9,3859.53
7,60.0,26.0,0.0,-1400.0,-100.0,-1200.0,-2000.0,0.0005,-85584.3,3669.63
8,60.0,26.0,0.0,-1200.0,-100.0,-1200.0,-2000.0,0.0005,-82955.1,3580.68
9,60.0,26.0,0.0,-1000.0,-100.0,-1200.0,-2000.0,0.0005,-79978.0,3467.62


In [11]:
using CSV
CSV.write("Heterogeneous ACW-MDP.csv", df)

"Heterogeneous ACW-MDP.csv"